In [1]:
import requests
import json
import pandas as pd
import os

In [15]:
api_key= os.environ.get('API_KEY_FDA')

In [3]:
### With an API key: 240 requests per minute, per key. 120,000 requests per day, per key.

In [6]:
foods_fda= requests.get(f'https://api.fda.gov/food/event.json?api_key={api_key}&search=date_started:[20110101+TO+20211231]&limit=1000')

In [7]:
foods_fda.status_code

200

In [8]:
base= foods_fda.json()
resultados= base['results']
flat_base= pd.json_normalize(resultados)
flat_base.shape

(1000, 9)

In [9]:
base_url = f'https://api.fda.gov/food/event.json?api_key={api_key}&search=date_started:[20110101+TO+20211231]&limit=1000'
food_data = pd.DataFrame()

response = requests.get(base_url)
total_count = response.json()['meta']['results']['total']

header = response.headers
df = pd.json_normalize(response.json()['results'])

while 'Link' in response.headers:
    next_page= response.headers['Link'].split(';')[0][1:-1] 
    response = requests.get(next_page)
    json_respuesta= response.json()
    json_resultados= json_respuesta['results']
    flat_data= pd.json_normalize(json_resultados)
    food_data=food_data.append(flat_data)
    
total_count

43270

In [11]:
final_fda=flat_base.append(food_data)
final_fda.shape

(43270, 9)

In [12]:
#Aqui solo compruebo que mis datos estan completos a partir de mi conteo total
print(total_count)

43270


In [13]:
final_fda.columns

Index(['report_number', 'outcomes', 'date_created', 'reactions',
       'date_started', 'products', 'consumer.age', 'consumer.age_unit',
       'consumer.gender'],
      dtype='object')

In [16]:
final_fda.head()

,report_number,outcomes,date_created,reactions,date_started,products,consumer.age,consumer.age_unit,consumer.gender
0,2020-CFS-011364,"[Death, Medically Important]",20201105,"[INJURY, DEATH]",20170401,"[{'role': 'SUSPECT', 'name_brand': 'EXEMPTION ...",56,year(s),F
1,2018-CFS-009718,[Medically Important],20180727,[COLITIS ISCHAEMIC],20180101,"[{'role': 'SUSPECT', 'name_brand': 'CITRACAL C...",79,year(s),F
2,2018-CFS-003276,[Medically Important],20180227,[OVARIAN CANCER],20120401,"[{'role': 'SUSPECT', 'name_brand': 'EXEMPTION ...",NaN,NaN,F
3,2019-CFS-004071,"[Disability, Medically Important]",20190419,"[MALAISE, VOMITING]",20190418,"[{'role': 'SUSPECT', 'name_brand': 'JAPANESE S...",8,month(s),F
4,2019-CFS-002461,"[Patient Visited Healthcare Provider, Other Se...",20190308,"[HEART RATE IRREGULAR, COSTOCHONDRITIS, EMERGE...",20181229,"[{'role': 'SUSPECT', 'name_brand': 'EYEPROMISE...",49,year(s),F


In [14]:
#Guardar en CSV
x=final_fda.to_csv('FDA_Food_10years.csv', index=False)